## Create projects

In [1]:
from morpheus.project.types.Permissions import Permissions, Visibility
from morpheus.common.types import DateTime
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.types.event_sourcing.EventEnvelope import EventEnvelope
from morpheus.project.types.Project import ProjectId, Project, Metadata, Name, Description
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.project.infrastructure.event_sourcing.ProjectEventBus import project_event_bus
from morpheus.common.types.identity.Identity import UserId

user_id_1 = UserId.new()
user_id_2 = UserId.new()
project_user_1_ids = [ProjectId.new(), ProjectId.new()]
project_user_2_ids = [ProjectId.new(), ProjectId.new(), ProjectId.new()]

# projects owned by user 1
for i, project_id in enumerate(project_user_1_ids):
    project = (Project.new(project_id=project_id, user_id=user_id_1)
                .with_updated_metadata(
                    Metadata.new()
                    .with_updated_name(Name.from_str(f'Notebook Project 1_{i}'))
                    .with_updated_description(Description.from_str(f'This is project 1_{i} created in a notebook'))
                )
              )
    project_created_event = ProjectCreatedEvent.from_project(project, DateTime.now())

    project_event_bus.record(
        EventEnvelope(event=project_created_event, metadata=EventMetadata.without_creator({'created_in_notebook': True}))
    )

# projects owned by user 2
for i, project_id in enumerate(project_user_2_ids):
    project = (Project.new(project_id=project_id, user_id=user_id_2)
                .with_updated_metadata(
                    Metadata.new()
                    .with_updated_name(Name.from_str(f'Notebook Project 2_{i}'))
                    .with_updated_description(Description.from_str(f'This is project 2_{i} created in a notebook'))
                )
              )
    project_created_event = ProjectCreatedEvent.from_project(project, DateTime.now())

    project_event_bus.record(
        EventEnvelope(event=project_created_event, metadata=EventMetadata.without_creator({'created_in_notebook': True}))
    )

# public project
public_project_owner_user_id = UserId.new()
public_project_id = ProjectId.new()
public_project = (Project.new(project_id=public_project_id, user_id=public_project_owner_user_id)
                .with_updated_metadata(
                    Metadata.new()
                    .with_updated_name(Name.from_str(f'Public Project'))
                    .with_updated_description(Description.from_str(f'This is a public project created in a notebook'))
                )
                .with_updated_permissions(
                    Permissions.new(public_project_owner_user_id).with_updated_visibility(Visibility.PUBLIC)
                )
              )
project_created_event = ProjectCreatedEvent.from_project(public_project, DateTime.now())
project_event_bus.record(
    EventEnvelope(event=project_created_event, metadata=EventMetadata.without_creator({'created_in_notebook': True}))
)


## Add members to project 1_1

In [2]:
from morpheus.project.types.Permissions import Role
from morpheus.project.domain.events.ProjectPermissionEvents.PermissionEvents import MemberAddedEvent

admin_id = UserId.new()
editor_id = UserId.new()
viewer_id = UserId.new()

project_id = project_user_1_ids[0]

admin_added_event = MemberAddedEvent.from_user_id_and_role(project_id, admin_id, Role.ADMIN, DateTime.now())
project_event_bus.record(EventEnvelope(event=admin_added_event, metadata=EventMetadata.without_creator({'created_in_notebook': True})))

editor_added_event = MemberAddedEvent.from_user_id_and_role(project_id, editor_id, Role.EDITOR, DateTime.now())
project_event_bus.record(EventEnvelope(event=editor_added_event, metadata=EventMetadata.without_creator({'created_in_notebook': True})))

viewer_added_event = MemberAddedEvent.from_user_id_and_role(project_id, viewer_id, Role.VIEWER, DateTime.now())
project_event_bus.record(EventEnvelope(event=viewer_added_event, metadata=EventMetadata.without_creator({'created_in_notebook': True})))

## Read projects

In [3]:
from morpheus.common.types.identity.Identity import Identity
from morpheus.project.application.read.ProjectReader import project_reader

def print_summaries(summaries):
    print([(summary.project_name.to_str(), summary.visibility.to_str(), [privilege.value for privilege in privileges]) for (summary, privileges) in summaries])

another_user = Identity(user_id=UserId.new(), group_ids=[], is_admin=False)
print('only public')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(another_user))

user1 = Identity(user_id=user_id_1, group_ids=[], is_admin=False)
print('user 1')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(user1))

user2 = Identity(user_id=user_id_2, group_ids=[], is_admin=False)
print('user 2')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(user2))

admin = Identity(user_id=admin_id, group_ids=[], is_admin=False)
print('admin')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(admin))

editor = Identity(user_id=editor_id, group_ids=[], is_admin=False)
print('editor')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(editor))

viewer = Identity(user_id=viewer_id, group_ids=[], is_admin=False)
print('viewer')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(viewer))

super_admin = Identity(user_id=UserId.new(), group_ids=[], is_admin=True)
print('super admin (sees all projects as owner)')
print_summaries(project_reader.get_project_summaries_with_user_privileges_for_identity(super_admin))

only public
[('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project'])]
user 1
[('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Notebook Project 1_0', 'private', ['full_access']), ('Notebook Project 1_1', 'private', ['full_access']), ('Public Project', 'public', ['view_project'])]
user 2
[('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Notebook Project 2_0', 'private', ['full_access']), ('Notebook Project 2_1', 'private', ['full_access']), ('Notebook Project 2_2', 'private', ['full_access']), ('Public Project', 'public', ['view_project'])]
admin
[('Public Project', 'public', ['view_project']), ('Public Project', 'public', ['view_project']), ('Public Project', 'pu

## Access to Event Log

In [5]:
from morpheus.common.types.Exceptions import InsufficientPermissionsException
from morpheus.project.types.permissions.Privilege import Privilege
from morpheus.project.application.read.PermissionsReader import permissions_reader

print('Only superadmin can view project event log for public project')
permissions_reader.assert_identity_can(Privilege.EDIT_PROJECT, super_admin, public_project_id)
for identity in [user1, user2, admin, editor, viewer]:
  try:
      permissions_reader.assert_identity_can(Privilege.EDIT_PROJECT, identity, public_project_id)
      raise Exception('Should have thrown exception')
  except InsufficientPermissionsException as e:
      print(e)

print('Only superadmin, owner, project admin and editor can view project event log for project 1_1')
for identity in [super_admin, user1, admin, editor]:
      permissions_reader.assert_identity_can(Privilege.EDIT_PROJECT, identity, project_user_1_ids[0])
for identity in [user2, viewer]:
  try:
      permissions_reader.assert_identity_can(Privilege.EDIT_PROJECT, identity, project_user_1_ids[0])
      raise Exception('Should have thrown exception')
  except InsufficientPermissionsException as e:
      print(e)

Only superadmin can view project event log for public project
User "93e08bbb-ae84-49a9-9f96-6931d8903f61" does not have privilege "edit_project" on project "5e0a2ec4-1698-4772-9474-bdd37aac9bfe"
User "e88bc6d8-81e8-4588-92df-8fb829e36123" does not have privilege "edit_project" on project "5e0a2ec4-1698-4772-9474-bdd37aac9bfe"
User "2b6e9cfa-a7c3-403c-ad4d-a398c1914072" does not have privilege "edit_project" on project "5e0a2ec4-1698-4772-9474-bdd37aac9bfe"
User "dd23c652-d64c-4658-9a09-d6ef90edbc74" does not have privilege "edit_project" on project "5e0a2ec4-1698-4772-9474-bdd37aac9bfe"
User "69e3dcc9-03de-43bc-b4d8-9569aceadb9d" does not have privilege "edit_project" on project "5e0a2ec4-1698-4772-9474-bdd37aac9bfe"
Only superadmin, owner, project admin and editor can view project event log for project 1_1
User "e88bc6d8-81e8-4588-92df-8fb829e36123" does not have privilege "edit_project" on project "5cb72255-a03e-453d-923e-8961ab2e6b4f"
User "69e3dcc9-03de-43bc-b4d8-9569aceadb9d" does

## Read Privileges

In [9]:
def print_privileges(identity, user_text, project_id):
    print(f'{user_text} can {[privilege.value for privilege in permissions_reader.get_privileges_for_identity(identity, project_id)]}')

print_privileges(super_admin, 'super_admin on public project', public_project_id)
print_privileges(user1, 'user with no role on public project', public_project_id)

print_privileges(super_admin, 'super_admin', project_user_1_ids[0])
print_privileges(user1, 'owner', project_user_1_ids[0])
print_privileges(admin, 'admin', project_user_1_ids[0])
print_privileges(editor, 'editor', project_user_1_ids[0])
print_privileges(viewer, 'viewer', project_user_1_ids[0])
print_privileges(user2, 'user with no role', project_user_1_ids[0])


super_admin on public project can ['full_access']
user with nor role on public project can ['view_project']
super_admin can ['full_access']
owner can ['full_access']
admin can ['view_project', 'edit_project', 'manage_project']
editor can ['view_project', 'edit_project']
viewer can ['view_project']
user with no role can []
